# Custom Kernels

In this tutorial we will learn:

 * How to define a custom Kernel.
 
 * How to use it in a *dcgpy.expression_double*.
 
 NOTE: when defining custom kernels directly via the python interface a slowdown is to be expected for two main reasons: a) python callables cannot be called from different threads (only processes) anb b) an added c++/python layer is added and forces conversions 

In [1]:
# Some necessary imports.
import dcgpy
from time import time
# Sympy is nice to have for basic symbolic manipulation.
from sympy import init_printing
from sympy.parsing.sympy_parser import *
init_printing()
# Fundamental for plotting.
from matplotlib import pyplot as plt
%matplotlib inline

### 1 - Write the nonlinearity and its symbolic form

In [2]:
# Lets define some non-linear function we would like to use as a computation unit in a dCGP:
def my_fun(x):
    return exp(-x[0]*x[0])

# We need also to define the textual form of such a kernel so that, for example, symbolic manipulators 
# can try to understand its semantic. In this function x will be a list of symbols like ["x", "y", "z"]
def my_fun_print(x):
    return "exp(-" + x[0] + "**2)"

# Note that its left to the user to define a extual representation that makes sense and is truthful, no checks are done.
# All symbolic manipulations will rely on the fact that such a representation makes sense.

In [3]:
# ... and see by example how these functions work:
from numpy import exp
a = my_fun([0.2,-0.12,-0.0011])
b = my_fun_print(["x", "y", "z"])
print(b + " is: " + str(a))

exp(-x**2) is: 0.9607894391523232


### 2 - Define a dcgp.kernel with our new callables


In [4]:
# Since the nonlinearities we wrote can operate on gduals as well as on double we can define 
# both a kernel_double and a kernel_gdual_double (here we will only use the first one)
my_kernel_double = dcgpy.kernel_double(my_fun, my_fun_print, "my_gaussian")
my_kernel_gdual_double = dcgpy.kernel_gdual_double(my_fun, my_fun_print, "my_gaussian")

In [5]:
a = my_kernel_double([0.2,-0.12,-0.0011])
b = my_kernel_double(["x", "y", "z"])
print(b + " is: " + str(a))

exp(-x**2) is: 0.9607894391523232


### 3 - Profiling the speed
Kernels defined in python introduce some slow down. Here we time 1000 calls to our exp(-x^2) function, when:

 * its wrapped in a python dcgpy.kernel
 * its coming from the shipped dcgpy package (cpp implementation)
 * its a normal python callable

In [6]:
cpp_kernel = dcgpy.kernel_set_double(["gaussian"])[0]

In [7]:
start = time()
_ = [my_kernel_double([i/1000, 0.3]) for i in range(1000)]
end =  time()
print("Elapsed (ms)", (end-start) * 1000)

Elapsed (ms) 8.867025375366211


In [8]:
start = time()
_ = [cpp_kernel([i/1000, 0.3]) for i in range(1000)]
end =  time()
print("Elapsed (ms)", (end-start) * 1000)

Elapsed (ms) 2.789735794067383


In [9]:
start = time()
_ = [my_fun([i/1000, 0.3]) for i in range(1000)]
end =  time()
print("Elapsed (ms)", (end-start) * 1000)

Elapsed (ms) 2.9599666595458984


### Using the new kernel in a dcpy.expression

In [10]:
ks = dcgpy.kernel_set_double(["sum", "mul", "diff"])
ks.push_back(my_kernel_double)
print(ks)

[sum, mul, diff, my_gaussian]


In [11]:
ex = dcgpy.expression_double(inputs=1,
                             outputs=1,
                             rows=1,
                             cols=6,
                             levels_back=6,
                             arity=2,
                             kernels=ks(),
                             n_eph=0,
                             seed = 39)
print(ex(["x"]))

['((exp(-(x-x)**2)+(x-x))-(x-x))']


In [12]:
# We use the expression internal simplify which is calling sympy. Since our symbolic representation
# of the Kernel is parsable by sympy, a simplified result is possible.
ex.simplify("x")